In [ ]:
import pandas as pd
from prepare_actuals import custom_date_parser
import numpy as np

In [ ]:
load_profiles_2016 = pd.read_csv('LoadProfile_20IPs_2016.csv', skiprows=1, delimiter=";", index_col=0, date_parser=custom_date_parser)
actuals = pd.read_csv('2016_actuals.csv', index_col=0, parse_dates=True)

forecasts = actuals.copy()
forecasts.loc[:, forecasts.columns != 'dataset_id'] = 0

In [ ]:
def the_fancy_forecaster(time_index_to_forecast, train_data):
    # This forecaster predicts the value which were observed 24h ago
    return train_data.loc[time_index_to_forecast - pd.Timedelta(days=1)].values

In [ ]:
TRAIN_DATA_LENGTH = pd.Timedelta(hours=24*7)

for dataset_id in actuals['dataset_id'].unique():
    actuals_i = actuals[actuals['dataset_id'] == dataset_id]
    for load in [x for x in actuals_i.columns if x != 'dataset_id']:
        actuals_i_j = actuals_i[load]
        start_of_test = actuals_i_j.index[0]
        train_data_i_j = load_profiles_2016[load][start_of_test - TRAIN_DATA_LENGTH:start_of_test - pd.Timedelta(minutes=15)]
        forecast = the_fancy_forecaster(actuals_i_j.index, train_data_i_j)
        forecasts.loc[actuals_i_j.index, load] = forecast

In [ ]:
def calculate_rmse(actual, predicted):
    return np.sqrt(((actual - predicted) ** 2).mean())

# Calculate RMSE for each pair of actual and predicted columns
rmse_results = {}

for col in [x for x in forecasts.columns if x != 'dataset_id']:
    forecast = forecasts[col]
    actual = actuals[col]
    rmse_results[col] = calculate_rmse(actual, forecast)

# Output RMSE for each column pair
print(rmse_results)